# Population Raster Exploration

This notebook is to explore the Global Human Settlement Layer Data.

1. First goal is to get the GHS-Mod (https://ghsl.jrc.ec.europa.eu/download.php?ds=smod) recategorized as rural & urban.
2. Maybe create a stacked raster with the GHS-Pop 2015 (https://ghsl.jrc.ec.europa.eu/ghs_pop2019.php) stacked on top of the urban/rural GHS-SMOD

Note, that the fils I am working with right now crs is in World Mollweide (EPSG:54009) at 1km spatial res.

Built with conda env geo37

By Cascade Tuholske 2019-08-09

#### Dependencies

In [1]:
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot

#### Open Files

In [2]:
# File Paths
DATA_IN = '../tuholske_data/raw/EU_JRC/'
DATA_OUT = '../tuholske_data/interim/'

In [3]:
# Files
pop2015_fn = 'GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif'
smod2015_fn = 'GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V1_0/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V1_0.tif'

In [4]:
pop2015 = rasterio.open(DATA_IN+pop2015_fn)
smod2015 = rasterio.open(DATA_IN+smod2015_fn)

#### Check Meta Data

In [ ]:
print(pop2015.meta)
print(smod2015.meta)

## Mask Smod

In [ ]:
def array_mask(arr, mask, value):
    """ Function masks arrays based on a boolian array of zeros and 1s
    """
    
    m = np.ma.masked_where(mask== value, arr)
    m = np.ma.filled(m, 0)
    
    return m

In [12]:
# get arrays

smod_arr = smod2015.read(1)

In [ ]:
# Look up SMOD classes 

# Classes 30 – 23 – 22 – 21 if aggregated form the “urban domain”, 13 – 12 – 11 form the “rural domain”
# 10 is WATER
# See GHS WhitePaper for info: https://ghsl.jrc.ec.europa.eu/documents/GHSL_Data_Package_2019.pdf?t=1478q532234372

np.unique(smod_arr)

In [13]:
# Make masks
# Codess: -200 = NaN, 10 = Water, 111 = Rural, 222 = Urban

# rural set to 1, urban set to 2
smod_arr[(smod_arr > 10) & (smod_arr < 21)] = 111  # RURAL 
smod_arr[(smod_arr >= 21) & (smod_arr < 111)] = 222 # URBAN 

print(np.unique(smod_arr))

[-200   10  111  222]


In [14]:
# Write out new urban/rural array as geo tif

def raster_write(meta, array, file_out):
    """ function to write out a raster file with an np array
    requires meta data for raster, np array & file out path and name
    """
    
    kwargs = meta

    # Update kwargs (change in data type)
    kwargs.update(dtype=rasterio.float32, count = 1)

    with rasterio.open(file_out, 'w', **kwargs) as dst:
        dst.write_band(1, array.astype(rasterio.float32))


In [15]:
meta = smod2015.meta
out_fn = smod2015_fn = DATA_OUT+'GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V1_0_Urban-Rural.tif'

In [16]:
# raster_write(meta, smod_arr, out_fn)